In [81]:
import numpy as np
import pandas as pd

In [82]:
data=pd.read_csv("titanic.csv")

In [83]:
data.head(n=10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [84]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 66.2+ KB


In [110]:
columns_to_drop=["PassengerId","Name","Ticket","Cabin","Embarked"]

data_clean=data.drop(columns_to_drop,axis=1)

In [111]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
dtypes: float64(2), int64(4), object(1)
memory usage: 45.3+ KB


In [112]:
data_clean=data_clean.fillna(data_clean["Age"].mean())

In [113]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
dtypes: float64(2), int64(4), object(1)
memory usage: 45.3+ KB


In [89]:
data_clean.loc[2]

Survived         1
Pclass           3
Sex         female
Age             26
SibSp            0
Parch            0
Fare         7.925
Name: 2, dtype: object

In [90]:
input_cols=["Pclass","Sex","Age","SibSp","Parch","Fare"]

output_cols=["Survived"]

In [91]:
X=data_clean[input_cols]
Y=data_clean[output_cols]

In [92]:
print(X.shape)

(891, 6)


In [93]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()

data_clean["Sex"]=le.fit_transform(data_clean["Sex"])

In [94]:
#define  Entropy and indo gain

In [95]:
def entropy(col):
    counts=np.unique(col,return_counts=True)
    N=float(col.shape[0])
    
    entropy=0.0
    
    for ix in counts[1]:
        p= ix/N
        entropy+= (-1.0*p*np.log2(p))
        
    return entropy

In [96]:
def divide_data(x_data,fkey,fval):
    
    x_right=pd.DataFrame([],columns=x_data.columns)
    x_left=pd.DataFrame([],columns=x_data.columns)
    
    for ix in range(x_data.shape[0]):
        val=x_data[fkey].loc[ix]
        
        if val>fval:
            x_right=x_right.append(x_data.loc[ix])
        else:
            x_left=x_left.append(x_data.loc[ix])
            
    
    return x_left,x_right

In [97]:
#col=np.array([1,1,1,1,0,0,0,0,2,2,2,2])
#entropy(col)

In [98]:
def info_gain(x_data,fkey,fval):
    #split the data in left and right
    
    left,right=divide_data(x_data,fkey,fval)
    
    #what percent of total samples on left and right
    
    l=float(left.shape[0]/x_data.shape[0])
    r=float(right.shape[0]/x_data.shape[0])
    
    #All examples in one side
    
    if left.shape[0]==0 or right.shape[0]==0:
        return -1000000
    
    i_gain=entropy(x_data.Survived)-(l*entropy(left.Survived))-(r*entropy(right.Survived))
    
    return i_gain

In [99]:
for fx in X.columns:
    print(fx)
    print(info_gain(data_clean,fx,data_clean[fx].mean()))

Pclass
0.07579362743608165
Sex
0.2176601066606143
Age
0.0008836151229467681
SibSp
0.009584541813400127
Parch
0.015380754493137666
Fare
0.04214069283899541


In [109]:
class  DecisionTree:
    #constructor
    
    def __init__(self,depth=0,max_depth=5):
        
        self.left=None
        self.right=None
        self.fkey=None
        self.fval=None
        self.max_depth=max_depth
        self.depth=depth
        self.target=None
        
    def train(self,X_train):
        
        features=['Pclass','Sex','Age','SibSp','Parch','Fare']
        
        info_gains=[]
        print("here")
        for ix in features:
            
            i_gain=info_gain(X_train,ix,X_train[ix].mean())
            info_gains.append(i_gain)
            
        self.fkey=features[np.argmax(info_gains)]
        print(self.fkey)
        self.fval=X_train[self.fkey].mean()
        
        #split data
        
        data_left,data_right=divide_data(X_train,self.fkey,self.fval)
        data_left=data_left.reset_index(drop=True)
        data_right=data_right.reset_index(drop=True)
        
        #truely a left node
        
        if(data_left.shape[0]==0 or data_right.shape[0]==0):
            if(X_train.Survived.mean()>=0.5):
                self.target="Survive"
            else:
                self.target="Dead"
                
            return
        
        #Recursive case
        
        self.left = DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.left.train(data_left)
        
        self.right = DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.right.train(data_right)
        
        #You can set the target at every node
        if X_train.Survived.mean() >= 0.5:
            self.target = "Survive"
        else:
            self.target = "Dead"
        return
    
    def predict(self,test):
        if test[self.fkey]>self.fval:
            #go to right
            if self.right is None:
                return self.target
            return self.right.predict(test)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)
                
        

            
        

In [107]:
dt = DecisionTree()
dt.train(data_clean)


here
Sex
here
Pclass
here
Pclass
here
Parch
here
Age
here
Age
here
Age
here
Age
here
Age
here
Age
here
Pclass
here
Fare
here
Pclass
here
Pclass
here
Pclass
here
Age
here
SibSp
here
Pclass
here
Pclass
here
Pclass
here
Age
here
Age
here
SibSp
here
Pclass
here
Pclass
here
Pclass
here
Age
here
Fare
here
Pclass
here
Fare
here
Pclass
here
Pclass
here
Pclass
here
Age
here
Age
here
Age
here
Pclass
here
SibSp
here
Fare
here
Pclass
here
Pclass
here
Fare
here
Fare
here
Fare
here
Pclass
here
Pclass
here
Fare
here
Pclass
here
Pclass
here
Fare
here
Pclass
here
Pclass
here
Age
here
Fare
here
Fare
here
Pclass
here
Pclass
here
Fare
here
Fare
here
Pclass
here
Pclass
here
Pclass
here
Pclass
here
Age
here
Age
here
Pclass
here
SibSp
here
Pclass
here
Fare
here
Pclass
here
Pclass
here
SibSp
here
Fare
here
Pclass
here
Pclass
here
Fare
here
Fare
here
Pclass
here
Pclass
here
Fare
here
Pclass
here
Pclass
here
Age
here
Age
here
Age
here
Age
here
Pclass
here
SibSp
here
Fare
here
Pclass
here
Pclass
here
Pclass
here

KeyboardInterrupt: 

In [108]:
print("ready")

ready
